In [ ]:
import io
from ipyfilechooser import FileChooser
import matplotlib.pyplot as plt
%matplotlib notebook
import numpy as np
from ipywidgets import *
from matplotlib.gridspec import GridSpec
import matplotlib.ticker
from Pre import *
from Post import *
from Calculate import *
from datetime import datetime

In [ ]:
print('This tool has three different calculation modes. First, please choose which mode you want to use.')


In [ ]:
print('Select your time series of precipitation and concentration input file:')
fc_c = FileChooser('')
display(fc_c)

In [ ]:
print('Select your sample concentrations input file:')
fc_s = FileChooser('')
display(fc_s)

In [ ]:
try:
    rain = Pre.convertTimeC(pd.read_csv(fc_c.selected, sep=';', header=None))
    sample = Pre.convertTimeS(pd.read_csv(fc_s.selected, sep=';', header=None))
except:
    print('An error occured!')    

In [ ]:
rain = Pre.checkZero(rain)

In [ ]:
rain = Pre.weightVolumes(rain)

In [ ]:
print('This is not integrated as an dynamic parameter. Set value once and do not change.')
print('Calculation in the vadose zone:')
Par()
def setVadoseZone(vadose_zone):
    if vadose_zone == 'No calculation':
        Par.SOILM = 0
    elif vadose_zone == 'Calculation based on monthly factor':
        print('To use this functionality, you must implement it manually')
        # Par.SOILM = 1
    elif vadose_zone == 'Calculation based on hydrologic model':
        # Par.SOILM = 2
        print('To use this functionality, you must implement it manually')
interact(setVadoseZone, vadose_zone = widgets.RadioButtons(options=['No calculation',
                                       'Calculation based on monthly factor',
                                       'Calculation based on hydrologic model'],
                              layout={'width': 'max-content'},
                              description='Calculation in the vadose zone:',
        style={'description_width': 'initial'}))

In [ ]:
if Par.SOILM == 0:
    print('There will be no pre processing regarding to the consideration of evapotranspiration processes.')
    Cin = rain['weighted_c']
elif Par.SOILM == 1:
    pass
elif Par.SOILM == 2:
    pass

In [ ]:
print('Here you can see your input datas and the sample values')
Par.n = 0
sample_vis = sample.drop(sample[sample[3] == 0].index)
fig = plt.figure(figsize=(8, 5), constrained_layout=True)
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)
ax1.plot(rain['Date'], rain[3], label='Input Tracer 1', color='green')
ax1.plot(sample_vis['Date'], sample_vis[1], visible=False)
ax1.legend()
ax2.plot(sample_vis['Date'], sample_vis[1], 'x')
ax1.set(title='Tracer Source',
        ylabel='$c$ [TU]',
        xlabel='t',
        ylim=0)
ax2.set(title='Tracer Sink',
        ylabel='$c$ [TU]',
        xlabel='t',
        ylim=0)
ax1.grid()
ax2.grid()

In [ ]:
print('All basic settings are now made. The settings that are now still necessary must be selected once at the beginning for the first model run. You then have the possibility to change the parameter combination and compare different variants.')

In [ ]:
print('Model type and parameter values:')
def setParameters(model, mean_tt, half_time, pd, eta):
    if model == 'Piston Flow Model':
        Par.MODNUM = 1
    elif model == 'Exponential Model':
        Par.MODNUM = 2
    elif model == 'Dispersion Model':
        Par.MODNUM = 3
    elif model == 'Linear Model':
        Par.MODNUM = 4
    elif model == 'Exponential - Piston Flow Model':
        Par.MODNUM = 5
    Par.TT = mean_tt
    Par.Thalf_1 = half_time
    Par.PD = pd
    Par.eta = eta
interact(setParameters,
    model = widgets.RadioButtons(options=['Piston Flow Model',
                                       'Exponential Model',
                                       'Dispersion Model',
                                       'Linear Model',
                                       'Exponential - Piston Flow Model'],
                              layout={'width': 'max-content'},
                              description='Modeltype:',
        style={'description_width': 'initial'}),
    mean_tt = widgets.BoundedIntText(value=10, min=1, max=100, step=1,
                                     description='Mean travel time [a]:',
                                    style={'description_width': 'initial'}),
    half_time = widgets.BoundedFloatText(value=10, min=1, max=100, step=0.001,
                                         description='Half Time [1/a]:',
                                        style={'description_width': 'initial'}),
    pd = widgets.BoundedFloatText(value=0.1, min=0.01, max=1, step=0.01,
                             description='PD:',
                                 style={'description_width': 'initial'}),
    eta = widgets.BoundedFloatText(value=0.5, min=0.01, max=1, step=0.01,
                             description='eta:',
                                  style={'description_width': 'initial'}))

In [ ]:
Par.step = 0
Par.n += 1
result = Tracer(Par, Cin, rain)
with open(r'out/log.txt', 'a') as f:
    f.write(str(datetime.now()))
    f.write(';' + str(Par.n))
    f.write(';' + str(Par.MODNUM))
    f.write(';' + str(Par.TT))
    f.write(';' + str(Par.Thalf_1))
    f.write(';' + str(Par.PD))
    f.write(';' + str(Par.eta))
    f.write('\n')
ax2.plot(rain['Date'], result.result, label='Run '+str(Par.n))
ax2.legend()
fig